# THE ORACLES

## REQUIREMENTS FOR THIS NOTEBOOK

In [ ]:
# Create a virtual environment with Python 

# Please, in order to properly run this notebook, make sure the following packages are installed

# qiskit                    0.45.0
# qiskit-aer                0.13.0
# qiskit-terra              0.45.0

# Once Python is installed and a new virtual environment is created, execute the following commands:
# pip install qiskit==0.45.0
# pip install quiskit-aer==0.13.0

## SOME AUXILIARY FUNCTIONS

### OBTAINING THE BINARY REPRESENTATION OF INTEGERS

In [6]:
import numpy as np

def to_binary(number, nbits=None):
    
    '''
    This fucntion transforms an integer to its binary form (string).
    If a determined number of bits is required (more than the needed ones),
    it can be passed as a parameter too, nbits, None by default.
    It is needed that the number of bits passed as a parameter is larger
    than the number of bits needed to write the number in binary. 

    Input:
    number: integer (int).
    nbits: integer (int), None by default

    Output:
    binary: string (str) containing the number in its binary form.
    It writes 0s in front if nbits is larger than the number of bits needed
    to write the binary form.
    '''

    if nbits is None:
        return bin(number)[2:]
    else:
        binary = bin(number)[2:]
        if nbits < len(binary):
            print('Error, nbits must be larger than %d.'%(len(binary)))
        else:
            return '0' * (nbits - len(binary)) + binary
        




In [ ]:
n = 3 #Integer for what the binary representation must be obtained
numbits = 4 #Precission for the representation (number of bits)
print (to_binary (n,numbits))

0011


### THE MULTICONTROLED Z GATE

In [8]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCXGate
from qiskit.extensions import UnitaryGate


def multi_control_z(nqubits):
  '''
  Function to create a multi-controlled Z gate.

  Input:
  nqubits: Integer (int) of the number of qubits in the gate (controls and target)
      This means that the gate has nqubits-1 controls and 1 target.

  Output:
  circuit: QuantumCircuit containing a multi-controlled Z gate.
    It has to be transformed with method .to_gate() to append to a QuantumCircuit larger.

  Example:

  main_circuit = QuantumCircuit(nqubits)

  gate_multi_z = multi_control_z(nqubits)

  main_circuit.append(gate_multi_z.to_gate(), range(nqubits))
  '''
  circuit=QuantumCircuit(nqubits,name=' CZ (%d)' %(nqubits))
  circuit.h(nqubits-1)
  gate = MCXGate(nqubits-1)
  circuit.append(gate, range(nqubits))
  circuit.h(nqubits-1)
  return circuit


In [9]:
nqubits = 6
circuit = multi_control_z(nqubits)
circuit.draw()

q_0: ───────■───────
            │       
q_1: ───────■───────
            │       
q_2: ───────■───────
            │       
q_3: ───────■───────
            │       
q_4: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_5: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘

### THE DIFFUSER

In [10]:
# Imports
from qiskit import QuantumCircuit


def diffuser_circuit(nqubits):
  '''
  Function to create the GROVER's diffuser circuit.

  Input:
  nqubits: Integer (int) of the number of qubits fo the circuit
  for wich the diffuser will be created.

  Output:
  circuit: QuantumCircuit containing the diffuser.
  It has to be transformed with method .to_gate() to append to a QuantumCircuit larger.

  Example:

  main_circuit = QuantumCircuit(nqubits)

  diffuser = difusser(nqubits)

  main_circuit.append(diffuser.to_gate(), range(nqubits-1, -1, -1))
  '''

  circuit=QuantumCircuit(nqubits,name=' Diffuser (%d)'%(nqubits))

  for qb in range (nqubits):
    circuit.h(qb)
  for qb in range (nqubits):
    circuit.x(qb)
  multi_z = multi_control_z(nqubits)
  circuit.append(multi_z.to_gate(),  range(nqubits-1, -1, -1))  
  for qb in range (nqubits):
    circuit.x(qb)
  for qb in range (nqubits):
    circuit.h(qb)
  return circuit

In [11]:
nqubits = 4
diffuser = diffuser_circuit(nqubits)
diffuser.draw()

┌───┐┌───┐┌──────────┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤3         ├┤ X ├┤ H ├
     ├───┤├───┤│          │├───┤├───┤
q_1: ┤ H ├┤ X ├┤2         ├┤ X ├┤ H ├
     ├───┤├───┤│   CZ (4) │├───┤├───┤
q_2: ┤ H ├┤ X ├┤1         ├┤ X ├┤ H ├
     ├───┤├───┤│          │├───┤├───┤
q_3: ┤ H ├┤ X ├┤0         ├┤ X ├┤ H ├
     └───┘└───┘└──────────┘└───┘└───┘

### GLOBAL PHASE

In [12]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCXGate
from qiskit.extensions import UnitaryGate


def globalphase():
    '''
    Function to create a gate the puts a global phase of Pi.
    The global phase can be applied afterwards to any qbit of a circuit

    Input:


    Output:
    circuit: QuantumCircuit containing the a sequence of gates that applied to any qbit of a circuit provides a 
    global phase of Pi to it
    It has to be transformed with method .to_gate() to append to a QuantumCircuit larger.

    Example:

    main_circuit = QuantumCircuit(nqubits)

    dg = globalphase(nqubits)

    main_circuit.append(globalphase.to_gate(), 0)
    '''
    circuit=QuantumCircuit(1,name=' GlobalPhase (%d)')

    circuit.z(0)
    circuit.x(0)
    circuit.z(0)
    circuit.x(0)
    

    return circuit

In [13]:
gb = globalphase()

gb.draw()


┌───┐┌───┐┌───┐┌───┐
q: ┤ Z ├┤ X ├┤ Z ├┤ X ├
   └───┘└───┘└───┘└───┘

## THE LESS_THAN ORACLE

In [14]:
def oracle_less_than(number, nqubits, name=None):

    '''
    This function builds a quantum circuit, an oracle, which marks with a pi-phase
    those states which represent numbers strictly smaler than the number given by parameter.

    The procedure is almost the same for all numbers, with the only exception of a difference
    if the first bit of the number in binary is 1 or 0.

    Input:
    number: integer (int) containing the objective number,
       or a string (str) with the binary representation of such number.
    nqubits: integer (int) number of qubits of the circuit.
       It must be larger than the number of digits of the binary representation of number.
    name: string (str), default None, name of the circuit.

    Output:
    circuit: QuantumCircuit which marks with fase pi the states which
    represent in binary the numbers strictly smaller than number.
    '''

    # Construction of the circuit
    if name:# If name is provided give such name to the circuit
        circuit = QuantumCircuit(nqubits, name=name)
    else: # Otherwise, the name is just " < number"
        circuit = QuantumCircuit(nqubits, name = ' < %d '%number)

    # Binary representation of the number
    num_binary = to_binary(number, nqubits)
    
    # Discard the 0s at the end, as they will not be used and save
    # unnecessary X gates
    num_binary = num_binary.rstrip('0')

    
    if num_binary[0] == '1':
        # If the first digit is 1
        # Mark all the states of the form |0q1...>
        circuit.x(nqubits-1)
        circuit.z(nqubits-1)
        circuit.x(nqubits-1)
    else:
        # If first digit is 0
        # Apply X gate to first qubit
        circuit.x(nqubits-1)
    
    # For loop on the remaining digits
    for position1, value in enumerate(num_binary[1:]):
        # Rename the position as it starts with 0 in the second bit and
        # we want it to be 1.
        position = position1 + 1
        #for nq in range(nqubits):
        #    circuit.barrier(nq)

        if value == '0':
            # If the digit is 0
            # Just apply a X gate
            circuit.x(nqubits-position-1)
        else:
            # If the digit bi is 1
            # Apply a multi-controlled Z gate to mark states of the shape:
            # |bn...bi+1 0 qi-1...q1>
            # where bn,...,bi+1 are the first n-i bits of m, which is of the shape bn...bi+1 1 bi-1...b1
            # because we just checked that bi is 1.
            # Hence, the numbers of the form bn...bi+1 0 qi-1...q1 are smaller than m.
            circuit.x(nqubits-position-1)
            multi_z = multi_control_z(position + 1)
            circuit.append(multi_z.to_gate(), range(nqubits-1, nqubits-position-2, -1))
            circuit.x(nqubits-position-1)
    
    for position, value in enumerate(num_binary):
        # Apply X gates to qubits in position of bits with a 0 value
        #for nq in range(nqubits):
        #    circuit.barrier(nq)
        if value == '0':
            circuit.x(nqubits-position-1)
        else:
            pass
    
    return circuit

In [15]:


nqubits = 4

# Less than oracle
number_less_than = 4
less_than_oracle = oracle_less_than(number=number_less_than, nqubits=nqubits)
less_than_oracle.draw()


q_0: ──────────────────────
                           
q_1: ──────────────────────
     ┌───┐┌──────────┐┌───┐
q_2: ┤ X ├┤1         ├┤ X ├
     ├───┤│   CZ (2) │├───┤
q_3: ┤ X ├┤0         ├┤ X ├
     └───┘└──────────┘└───┘

## THE GREATER_THAN ORACLE

In [16]:
def oracle_greater_than(number, nqubits, name=None):

    '''
    This function builds a quantum circuit, an oracle, which marks with a pi-phase
    those states which represent numbers strictly grater than the number given by parameter.

    The procedure is almost the same for all numbers, with the only exception of a difference
    if the first bit of the number in binary is 1 or 0.

    Input:
    number: integer (int) containing the objective number,
       or a string (str) with the binary representation of such number.
    nqubits: integer (int) number of qubits of the circuit.
       It must be larger than the number of digits of the binary representation of number.
    name: string (str), default None, name of the circuit.

    Output:
    circuit: QuantumCircuit which marks with fase pi the states which
    represent in binary the numbers strictly greatersmaller than number.

    This oracle relais on the use of the less_tan oracle levaraging the propertry:
    gretar_than (x) = less_than (x+1) + globelphase
    '''

    # Construction of the circuit
    if name:# If name is provided give such name to the circuit
        circuit = QuantumCircuit(nqubits, name=name)
    else: # Otherwise, the name is just " < number"
        circuit = QuantumCircuit(nqubits, name = ' > %d '%number)

    if number < (2**nqubits): # if number is not the greater namber that can be represented using nqubits
        number=number+1

    less_than = oracle_less_than(number=number, nqubits=nqubits)
    gp = globalphase()
    circuit.append(less_than.to_gate(),  range(0,nqubits, 1))
    #circuit.append(less_than.to_gate(),  range(nqubits-1, -1, -1))  
    circuit.append(gp.to_gate(), range(0, -1, -1))
    return circuit


In [17]:


nqubits = 7

# Greater than oracle
number_greater_than = 11
greater_than = oracle_greater_than(number=number_greater_than, nqubits=nqubits)
greater_than.draw()

┌─────────┐┌───────────────────┐
q_0: ┤0        ├┤  GlobalPhase (%d) ├
     │         │└───────────────────┘
q_1: ┤1        ├─────────────────────
     │         │                     
q_2: ┤2        ├─────────────────────
     │         │                     
q_3: ┤3  < 12  ├─────────────────────
     │         │                     
q_4: ┤4        ├─────────────────────
     │         │                     
q_5: ┤5        ├─────────────────────
     │         │                     
q_6: ┤6        ├─────────────────────
     └─────────┘

## THE RANGE_OF ORACLE

In [18]:
def oracle_range_of(lower,upper, nqubits, name=None):

    '''
    This function builds a quantum circuit, an oracle, which marks with a pi-phase
    those states which represent numbers strictly grater than lower and stricly less than upper.

    This oracle relais on the use of the less_than and greater_than oracles

    Input:
    lower: lower limit of the range.
    upper: upper limet of the range
    nqubits: integer (int) number of qubits of the circuit.
       It must be larger than the number of digits of the binary representation of number.
    name: string (str), default None, name of the circuit.

    Output:
    circuit: QuantumCircuit which marks with fase pi the states which
    represent in binary the numbers strictly greater and less than lower and upper respectively.

    '''

    # Construction of the circuit
    if name:# If name is provided give such name to the circuit
        circuit = QuantumCircuit(nqubits, name=name)
    else: 
        circuit = QuantumCircuit(nqubits, name = ' range_of  ')
    less=oracle_less_than(upper,nqubits)
    greater=oracle_greater_than(lower,nqubits)
    gp = globalphase()
    circuit.append(greater.to_gate(),  range(0, nqubits, 1)) 
    circuit.append(less.to_gate(),  range(0, nqubits, 1)) 
    #range(nqubits-1, -1, -1)
    circuit.append(gp.to_gate(), range(0, -1, -1))
    return circuit

In [19]:
nqubits = 4

# Greater than oracle
upper = 4
lower = 1
range_of = oracle_range_of(lower=lower, upper=upper, nqubits=nqubits)
range_of.draw()

┌────────┐┌────────┐┌───────────────────┐
q_0: ┤0       ├┤0       ├┤  GlobalPhase (%d) ├
     │        ││        │└───────────────────┘
q_1: ┤1       ├┤1       ├─────────────────────
     │   > 1  ││   < 4  │                     
q_2: ┤2       ├┤2       ├─────────────────────
     │        ││        │                     
q_3: ┤3       ├┤3       ├─────────────────────
     └────────┘└────────┘

## SOME SIMULATIONS

### LESS_THAN ORACLE SIMULATION

In [20]:
from math import sqrt

# Some examples:
method = "statevector"
sim = AerSimulator(method = method)

def program_less_than (nqubits,number_less_than):
  # This program creates the circuit of a qiskit program that produces a quantum states of nqubits qbits
  # where all the values of the state less than number_less_than have the greater amplitude
  qreg = QuantumRegister(nqubits)
  creg = ClassicalRegister(nqubits)
  qprogram = QuantumCircuit(qreg,creg)
  qprogram.h(qreg)
  diffuser = diffuser_circuit(nqubits)
  less_than_oracle = oracle_less_than(number=number_less_than, nqubits=nqubits)

  sq = 2**nqubits
  fr=sq/(2**nqubits-number_less_than)
  numit=round(fr)
  if numit == 0:
    numit=1
  #numit=6
  print (numit)
  for i in range(numit):
    qprogram.append(less_than_oracle.to_gate(), range(0,nqubits,1))
    qprogram.append(diffuser.to_gate(), range(0,nqubits,1))
    #qprogram.append(less_than_oracle.to_gate(), range(nqubits-1,-1,-1))
    #qprogram.append(diffuser.to_gate(), range(nqubits-1,-1,-1))
  qprogram.measure(qreg,creg)
  return qprogram


nqubits = 4
number_less_than = 4

qp = program_less_than(nqubits, number_less_than)
qp.draw()

transpiled_circuit = transpile(qp, backend = sim)

nshots = 200
sim_run = sim.run(transpiled_circuit, shots = nshots)
sim_result=sim_run.result()
counts_result = sim_result.get_counts(qp)

print('''Printing the various results followed by how many times they happened (out of the {} cases):\n'''.format(nshots), flush = True)
for i in range(len(counts_result)):
  print('-> Result \"{0}\" happened {1} times out of {2}'.format(
  list(sim_result.get_counts().keys())[i],
  list(sim_result.get_counts().values())[i],nshots), 
  flush = True)
qp.draw()



1
Printing the various results followed by how many times they happened (out of the 200 cases):

-> Result "0001" happened 55 times out of 200
-> Result "0000" happened 50 times out of 200
-> Result "0011" happened 39 times out of 200
-> Result "0010" happened 56 times out of 200


┌───┐┌────────┐┌────────────────┐┌─┐         
q3_0: ┤ H ├┤0       ├┤0               ├┤M├─────────
      ├───┤│        ││                │└╥┘┌─┐      
q3_1: ┤ H ├┤1       ├┤1               ├─╫─┤M├──────
      ├───┤│   < 4  ││   Diffuser (4) │ ║ └╥┘┌─┐   
q3_2: ┤ H ├┤2       ├┤2               ├─╫──╫─┤M├───
      ├───┤│        ││                │ ║  ║ └╥┘┌─┐
q3_3: ┤ H ├┤3       ├┤3               ├─╫──╫──╫─┤M├
      └───┘└────────┘└────────────────┘ ║  ║  ║ └╥┘
c2: 4/══════════════════════════════════╩══╩══╩══╩═
                                        0  1  2  3

### GREATER_THAN ORACLE SIMULATION

In [21]:
from math import sqrt

# Some examples:
method = "statevector"
sim = AerSimulator(method = method)

def program_greater_than (nqubits,number_greater_than):
  # This program creates the circuit of a qiskit program that produces a quantum states of nqubits qbits
  # where all the values of the state less than number_less_than have the greater amplitude
  qreg = QuantumRegister(nqubits)
  creg = ClassicalRegister(nqubits)
  qprogram = QuantumCircuit(qreg,creg)
  qprogram.h(qreg)
  diffuser = diffuser_circuit(nqubits)
  greater_than_oracle = oracle_greater_than(number=number_greater_than, nqubits=nqubits)

  sq = 2**nqubits
  fr=sq/(2**nqubits-number_greater_than)
  numit=round(fr)+1
  if numit == 0:
    numit=1
  #numit=6
  print (numit)
  for i in range(numit):
    qprogram.append(greater_than_oracle.to_gate(), range(0,nqubits,1))
    qprogram.append(diffuser.to_gate(), range(0,nqubits,1))
    #qprogram.append(greater_than_oracle.to_gate(), range(nqubits-1,-1,-1))
    #qprogram.append(diffuser.to_gate(), range(nqubits-1,-1,-1))
  qprogram.measure(qreg,creg)
  return qprogram


nqubits = 4
number_greater_than = 11

qp = program_greater_than(nqubits, number_greater_than)
qp.draw()

transpiled_circuit = transpile(qp, backend = sim)

nshots = 200
sim_run = sim.run(transpiled_circuit, shots = nshots)
sim_result=sim_run.result()
counts_result = sim_result.get_counts(qp)

print('''Printing the various results followed by how many times they happened (out of the {} cases):\n'''.format(nshots), flush = True)
for i in range(len(counts_result)):
  print('-> Result \"{0}\" happened {1} times out of {2}'.format(
  list(sim_result.get_counts().keys())[i],
  list(sim_result.get_counts().values())[i],nshots), 
  flush = True)
qp.draw()

4
Printing the various results followed by how many times they happened (out of the 200 cases):

-> Result "1101" happened 59 times out of 200
-> Result "1100" happened 46 times out of 200
-> Result "1111" happened 54 times out of 200
-> Result "1110" happened 41 times out of 200


┌───┐┌─────────┐┌────────────────┐┌─────────┐┌────────────────┐»
q9_0: ┤ H ├┤0        ├┤0               ├┤0        ├┤0               ├»
      ├───┤│         ││                ││         ││                │»
q9_1: ┤ H ├┤1        ├┤1               ├┤1        ├┤1               ├»
      ├───┤│   > 11  ││   Diffuser (4) ││   > 11  ││   Diffuser (4) │»
q9_2: ┤ H ├┤2        ├┤2               ├┤2        ├┤2               ├»
      ├───┤│         ││                ││         ││                │»
q9_3: ┤ H ├┤3        ├┤3               ├┤3        ├┤3               ├»
      └───┘└─────────┘└────────────────┘└─────────┘└────────────────┘»
c3: 4/═══════════════════════════════════════════════════════════════»
                                                                     »
«      ┌─────────┐┌────────────────┐┌─────────┐┌────────────────┐┌─┐         
«q9_0: ┤0        ├┤0               ├┤0        ├┤0               ├┤M├─────────
«      │         ││                ││         ││                │└╥┘┌─┐      
«q9_1: ┤1        ├┤1               ├┤1        ├┤1               ├─╫─┤M├──────
«      │   > 11  ││   Diffuser (4) ││   > 11  ││   Diffuser (4) │ ║ └╥┘┌─┐   
«q9_2: ┤2        ├┤2               ├┤2        ├┤2               ├─╫──╫─┤M├───
«      │         ││                ││         ││                │ ║  ║ └╥┘┌─┐
«q9_3: ┤3        ├┤3               ├┤3        ├┤3               ├─╫──╫──╫─┤M├
«      └─────────┘└────────────────┘└─────────┘└────────────────┘ ║  ║  ║ └╥┘
«c3: 4/═══════════════════════════════════════════════════════════╩══╩══╩══╩═
«                                                                 0  1  2  3

### RANGE_OF ORACLE SIMULATION

In [22]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, IBMQ, transpile
from qiskit.circuit.library import QFT
from qiskit.extensions import UnitaryGate
from math import sqrt

# Some examples:
method = "statevector"
sim = AerSimulator(method = method)

def program_range_of (nqubits,number_lower,number_upper):
  # This program creates the circuit of a qiskit program that produces a quantum states of nqubits qbits
  # where all the values of the state less than number_less_than have the greater amplitude
  qreg = QuantumRegister(nqubits)
  creg = ClassicalRegister(nqubits)
  qprogram = QuantumCircuit(qreg,creg)
  qprogram.h(qreg)
  diffuser = diffuser_circuit(nqubits)
  range_of_oracle = oracle_range_of(lower=number_lower, upper=number_upper, nqubits=nqubits)

  sq = 2**nqubits
  fr=sq/(number_upper-number_lower)
  numit=round(fr)+1
  if numit == 0:
    numit=1
  #numit=6
  print (numit)
  for i in range(numit):
    qprogram.append(range_of_oracle.to_gate(), range(0, nqubits, 1))
    qprogram.append(diffuser.to_gate(), range(0, nqubits, 1))
  qprogram.measure(qreg,creg)
  return qprogram


nqubits = 4
number_lower = 1
number_upper = 4

qp = program_range_of(nqubits, number_lower, number_upper)
qp.draw()

transpiled_circuit = transpile(qp, backend = sim)

nshots = 200
sim_run = sim.run(transpiled_circuit, shots = nshots)
sim_result=sim_run.result()
counts_result = sim_result.get_counts(qp)

print('''Printing the various results followed by how many times they happened (out of the {} cases):\n'''.format(nshots), flush = True)
for i in range(len(counts_result)):
  print('-> Result \"{0}\" happened {1} times out of {2}'.format(
  list(sim_result.get_counts().keys())[i],
  list(sim_result.get_counts().values())[i],nshots), 
  flush = True)
qp.draw()

6
Printing the various results followed by how many times they happened (out of the 200 cases):

-> Result "0011" happened 101 times out of 200
-> Result "0010" happened 99 times out of 200


┌───┐┌──────────────┐┌────────────────┐┌──────────────┐»
q16_0: ┤ H ├┤0             ├┤0               ├┤0             ├»
       ├───┤│              ││                ││              │»
q16_1: ┤ H ├┤1             ├┤1               ├┤1             ├»
       ├───┤│   range_of   ││   Diffuser (4) ││   range_of   │»
q16_2: ┤ H ├┤2             ├┤2               ├┤2             ├»
       ├───┤│              ││                ││              │»
q16_3: ┤ H ├┤3             ├┤3               ├┤3             ├»
       └───┘└──────────────┘└────────────────┘└──────────────┘»
 c4: 4/═══════════════════════════════════════════════════════»
                                                              »
«       ┌────────────────┐┌──────────────┐┌────────────────┐┌──────────────┐»
«q16_0: ┤0               ├┤0             ├┤0               ├┤0             ├»
«       │                ││              ││                ││              │»
«q16_1: ┤1               ├┤1             ├┤1               ├┤1             ├»
«       │   Diffuser (4) ││   range_of   ││   Diffuser (4) ││   range_of   │»
«q16_2: ┤2               ├┤2             ├┤2               ├┤2             ├»
«       │                ││              ││                ││              │»
«q16_3: ┤3               ├┤3             ├┤3               ├┤3             ├»
«       └────────────────┘└──────────────┘└────────────────┘└──────────────┘»
« c4: 4/════════════════════════════════════════════════════════════════════»
«                                                                           »
«       ┌────────────────┐┌──────────────┐┌────────────────┐┌──────────────┐»
«q16_0: ┤0               ├┤0             ├┤0               ├┤0             ├»
«       │                ││              ││                ││              │»
«q16_1: ┤1               ├┤1             ├┤1               ├┤1             ├»
«       │   Diffuser (4) ││   range_of   ││   Diffuser (4) ││   range_of   │»
«q16_2: ┤2               ├┤2             ├┤2               ├┤2             ├»
«       │                ││              ││                ││              │»
«q16_3: ┤3               ├┤3             ├┤3               ├┤3             ├»
«       └────────────────┘└──────────────┘└────────────────┘└──────────────┘»
« c4: 4/════════════════════════════════════════════════════════════════════»
«                                                                           »
«       ┌────────────────┐┌─┐         
«q16_0: ┤0               ├┤M├─────────
«       │                │└╥┘┌─┐      
«q16_1: ┤1               ├─╫─┤M├──────
«       │   Diffuser (4) │ ║ └╥┘┌─┐   
«q16_2: ┤2               ├─╫──╫─┤M├───
«       │                │ ║  ║ └╥┘┌─┐
«q16_3: ┤3               ├─╫──╫──╫─┤M├
«       └────────────────┘ ║  ║  ║ └╥┘
« c4: 4/═══════════════════╩══╩══╩══╩═
«                          0  1  2  3